### 3.1.3 Using the techniques you applied in Assignment #1, apply a masking or transformation mechanism to modify the detected PII elements and substitute with suitable replacements.
In the following section, we will apply techniques similar to those used in Assignment #1 to mask or transform Personally Identifiable Information (PII) detected in a dataset. The goal is to substitute these sensitive elements with suitable replacements while maintaining the overall structure and coherence of the data. 

We start by anonymizing the eight categories with help from the Faker library:
- PERSON: Replaces names of people with fake names generated by Faker.
- GPE (Geopolitical Entities): Substitutes names of countries, cities, states, etc., with random city names using Faker.
- DATE: Transforms dates into random dates. Specific dates like 'today', 'tomorrow', or 'next week' are replaced with dates that correspond to these descriptions.
- ORG (Organizations): Changes names of organizations, companies, agencies, etc., to random company names generated by Faker.
- NORP (Nationalities, Religious or Political Groups): Replaces nationalities, religions, and political group names with random country names, implying a change in nationality.
- CARDINAL (Numerals): Alters numerical values to be close to the original number but not exact, within a ±10% range or ±3, whichever is greater.
- ORDINAL (Ordinal Numbers): Generates random ordinal numbers (like 1st, 2nd, 3rd, etc.) to replace existing ordinal numbers.
- TIME: Changes time mentions to random times. Specific times of the day like 'morning' or 'evening' are replaced with times corresponding to those periods.

In [1]:
import pandas as pd
df = pd.read_csv("PII_tweet_emotions.csv")

In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [3]:
from datetime import datetime, timedelta
import random
from faker import Faker
import re

fake = Faker()

def close_number(original_number):
    try:
        num = int(original_number)
        # Generate a number within ±10% of the original number or ±3, whichever is greater
        percentage_variation = int(num * 0.1)
        min_variation = 3  # Minimum variation
        variation = max(min_variation, percentage_variation)
        return str(random.randint(max(0, num - variation), num + variation))
    except ValueError:
        # Return the original number if it's not an integer
        return original_number
    
def fake_ordinal():
    number = fake.random_int(min=1, max=100)
    suffix = ["th", "st", "nd", "rd"] + ["th"] * 6
    return str(number) + suffix[number % 10 if number % 100 not in [11, 12, 13] else 0]
    
def replace_date(entity_text):
    today = datetime.today()
    if entity_text.lower() in ['today']:
        new_date = fake.date_between(start_date=today, end_date=today)
    elif entity_text.lower() in ['tomorrow']:
        new_date = fake.date_between(start_date=today + timedelta(days=1), end_date=today + timedelta(days=1))
    elif entity_text.lower() in ['next week']:
        new_date = fake.date_between(start_date=today + timedelta(days=7), end_date=today + timedelta(days=14))
    # Add more conditions for other specific date phrases
    else:
        # For general date entities, return a random future date
        new_date = fake.future_date()
    
    return new_date.strftime("%Y-%m-%d")  # Convert the date to a string


def replace_time(entity_text):
    # Define time ranges
    morning_times = [f"{hour:02d}:{minute:02d} AM" for hour in range(6, 12) for minute in range(0, 60)]
    evening_times = [f"{hour:02d}:{minute:02d} PM" for hour in range(6, 12) for minute in range(0, 60)]

    if entity_text.lower() in ['morning']:
        return random.choice(morning_times)
    elif entity_text.lower() in ['evening', 'tonight']:
        return random.choice(evening_times)
    else:
        # For general time entities, return any random time
        return fake.time()

def replace_pii_with_fake(text):
    # Replace Twitter @username with @ followed by a fake first name
    text = re.sub(r'@(\w+)', lambda x: '@' + fake.first_name(), text)

    # Process the text using spaCy to identify named entities
    doc = nlp(text)

    # Iterate over the identified entities
    for ent in doc.ents:
        # Replace with fake data based on the entity type
        if ent.label_ == 'PERSON':
            text = re.sub(re.escape(ent.text), fake.name(), text)
        elif ent.label_ == 'GPE':
            text = re.sub(re.escape(ent.text), fake.city(), text)
        elif ent.label_ == 'DATE':
            text = re.sub(re.escape(ent.text), replace_date(ent.text), text)
        elif ent.label_ == 'ORG':
            text = re.sub(re.escape(ent.text), fake.company(), text)
        elif ent.label_ == 'NORP':
            text = re.sub(re.escape(ent.text), fake.country(), text)
        elif ent.label_ == 'CARDINAL':
            text = re.sub(re.escape(ent.text), lambda x: close_number(ent.text), text)
        elif ent.label_ == 'ORDINAL':
            text = re.sub(re.escape(ent.text), fake_ordinal(), text)
        elif ent.label_ == 'TIME':
            text = re.sub(re.escape(ent.text), replace_time(ent.text), text)
    return text

# Apply the function to the DataFrame
df['content'] = df['content'].apply(replace_pii_with_fake)

# Save the modified DataFrame to a new CSV file
df.to_csv("Anonymized_PII_tweet_emotions.csv", index=False)

That being done, we take a look at our anonymized dataset:

In [4]:
df = pd.read_csv("Anonymized_PII_tweet_emotions.csv")
print(df.iloc[:8])

     tweet_id   sentiment                                            content  \
0  1956967341       empty  Randolph, Green and Smith i know  i was listen...   
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...   
2  1956967696     sadness            Funeral ceremony...gloomy 2024-02-05...   
3  1956967789  enthusiasm               wants to hang out with friends SOON!   
4  1956968416     neutral  @Ellen We want to trade with someone who has S...   
5  1956968477       worry  Re-pinging @David: why didn't you go to prom? ...   
6  1956968487     sadness  I should be sleep, but im not! thinking about ...   
7  1956968636       worry               Hmmm. http://www.djhero.com/ is down   

                           PII  
0  [('@tiffanylue', 'PERSON')]  
1                           []  
2         [('friday', 'DATE')]  
3                           []  
4         [('Houston', 'GPE')]  
5           [('BC', 'PERSON')]  
6          [('2', 'CARDINAL')]  
7              

### 3.1.4 Analyse the text to determine if any information can be obtained after the transformation process. What conclusions can you draw from this?

The code is designed to first calculate semantic similarity between texts in an original dataset and their anonymized counterparts, assessing how well the anonymization process has retained the original text's meaning. Following this, it aims to check whether any Personally Identifiable Information (PII) from the original dataset remains in the anonymized dataset, ensuring the effectiveness of the anonymization in protecting privacy.

In [6]:
import pandas as pd
import torch
from scipy.spatial.distance import cosine
import numpy as np
from transformers import MobileBertTokenizer, MobileBertModel


# Load your datasets
df = pd.read_csv("tweet_emotions.csv")  # Make sure you've loaded the original dataset into 'df'
df_anonymized = pd.read_csv("Anonymized_PII_tweet_emotions.csv")

# Set device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize tokenizer and model
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model = MobileBertModel.from_pretrained('google/mobilebert-uncased').to(device)


# Modify the get_embedding function to send inputs to the GPU
def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# Function to calculate semantic similarity
def semantic_similarity(text1, text2, tokenizer, model):
    emb1 = get_embedding(text1, tokenizer, model)
    emb2 = get_embedding(text2, tokenizer, model)
    # Ensure embeddings are 1-D
    emb1 = np.squeeze(emb1)
    emb2 = np.squeeze(emb2)
    #print(text1 + " and " + text2)
    return 1 - cosine(emb1, emb2)

# Calculate similarities
try:
    similarity_scores = [semantic_similarity(orig, anon, tokenizer, model) for orig, anon in zip(df['content'], df_anonymized['content'])]
    df_anonymized['scores'] = similarity_scores
except ValueError as e:
    print(f"Error calculating similarity: {e}")

KeyboardInterrupt: 

Still not quite ready yet, needs some fixing 

In [12]:
import pandas as pd #
import spacy

# Load your datasets
original_dataset = pd.read_csv('PII_tweet_emotions.csv')
anonymized_dataset = pd.read_csv('Anonymized_PII_tweet_emotions.csv')

# Load spaCy's model
nlp = spacy.load('en_core_web_sm')  

def extract_text_entities(text, nlp_model):
    doc = nlp_model(text)
    return {ent.text for ent in doc.ents}

# Extract and store PII for both datasets
original_dataset['PII_original'] = original_dataset['content'].apply(lambda x: list(extract_text_entities(x, nlp)))
anonymized_dataset['PII_anonymised'] = anonymized_dataset['content'].apply(lambda x: list(extract_text_entities(x, nlp)))

# Merge the PII columns into the anonymized dataset
merged_dataset = anonymized_dataset.copy()
merged_dataset['PII_original'] = original_dataset['PII_original']

# Initialize the column for matches
merged_dataset['pii_matches'] = [[] for _ in range(len(merged_dataset))]

# Compare PII_original with PII_anonymised
for index, row in merged_dataset.iterrows():
    original_entities = set(row['PII_original'])
    anonymized_entities = set(row['PII_anonymised'])
    
    matches = original_entities.intersection(anonymized_entities)
    if matches:
        merged_dataset.at[index, 'pii_matches'] = list(matches)
        print(f"Row {index}: Found matches in anonymized data - {matches}")

# Save the updated merged dataset to a new CSV file
merged_dataset.to_csv('merged_dataset_with_pii_comparison.csv', index=False)


Row 21: Found matches in anonymized data - {'7am'}
Row 22: Found matches in anonymized data - {'#', 'the Balisage Markup Conference'}
Row 30: Found matches in anonymized data - {'the middle of the night'}
Row 34: Found matches in anonymized data - {'tomorrow'}
Row 35: Found matches in anonymized data - {'Screw'}
Row 42: Found matches in anonymized data - {'Blackberry'}
Row 53: Found matches in anonymized data - {'6hrs'}
Row 56: Found matches in anonymized data - {'B4', 'sorta'}
Row 62: Found matches in anonymized data - {'noisy&quot'}
Row 75: Found matches in anonymized data - {'Voobys'}
Row 84: Found matches in anonymized data - {'GB', '4', 'USB'}
Row 86: Found matches in anonymized data - {'one'}
Row 89: Found matches in anonymized data - {'one', 'iknow)&quot'}
Row 91: Found matches in anonymized data - {'Windows'}
Row 92: Found matches in anonymized data - {'strawberry'}
Row 107: Found matches in anonymized data - {'VIRUS'}
Row 108: Found matches in anonymized data - {'ahh!'}
Row 12